In [ ]:
import random
import os
from tqdm import tqdm
import sys
import joblib
import seaborn as sns
import pandas as pd
from scipy import interpolate
from scipy.spatial import distance
from sklearn import metrics
from scipy.stats import ttest_ind, ttest_1samp
from pymovements.gaze.transforms import pix2deg, pos2vel

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import argparse


from tensorflow.keras.models import Model
from sp_eyegan.preprocessing import data_loader as data_loader
from sp_eyegan.model import contrastive_learner as contrastive_learner
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV



%pylab inline
%load_ext autoreload
%autoreload 2

In [2]:
# params
dst_dir = 'results/'

In [3]:
adhd_files =  ['EKYT',
                 'ekyt_random_window_size_600_sd_0.05_sd_factor_1.25_embedding_size_128_stimulus_video_model_random_-1baseline_120',
                 'CLRGAZE',
                 'clrgaze_random_window_size_600_sd_0.05_sd_factor_1.25_embedding_size_512_stimulus_video_model_random_-1baseline_120',
                ]

In [4]:
name_mapping = {'clrgaze':'CLRGaze',
                'ekyt':'EKYT',
                'CLRGAZE':'CLRGaze',
                'EKYT':'EKYT',
                }

decimals = 3

videos = ['Video_Fractals',
          'Video_The_Present',
          'Video_Despicable_Me',
          'Video_Diary_of_a_Wimpy_Kid_Trailer',
         ]

for video in videos:
    print('evaluate ' + video)
    model_result_dicts = dict()
    for file in adhd_files:
        file = file + '_adhd_video_' + video + '.joblib'
        cur_path = dst_dir + file
        cur_model = name_mapping[file.split('_')[0]]
        try:
            cur_data = joblib.load(cur_path)
        except:
            continue
        auc_rounds_rf = cur_data['auc_rounds_rf']
        auc_rounds_nn = cur_data['auc_rounds_nn']
        if 'embedding' in file:
            cur_model = cur_model + ' (CP SP-EyeGAN) fine-tuning'
        else:
            cur_model = cur_model + ' (w/o pre-training)'
        model_result_dicts[cur_model] = np.array(auc_rounds_nn).ravel()
        better_random_pvalue = ttest_1samp(a=np.array(auc_rounds_nn).ravel(),popmean=.5,alternative = 'greater').pvalue
        if better_random_pvalue < 0.05:
            cur_p_value_add_str = '*'
        else:
            cur_p_value_add_str = ''
        cur_p_value_model_str = ''
        if 'w/o pre-training' not in cur_model:
            base_values = model_result_dicts[cur_model.replace('(CP SP-EyeGAN) fine-tuning','(w/o pre-training)')]
            tt_test_pvalue = ttest_ind(base_values,np.array(auc_rounds_nn).ravel(),alternative='two-sided').pvalue
            if tt_test_pvalue < 0.05 and np.mean(base_values) < np.mean(auc_rounds_nn):
                cur_p_value_model_str += '$\dagger$'
        print('    ' + cur_model + ' & ' + str(np.round(np.mean(auc_rounds_nn), decimals = decimals)) +\
             ' $\pm$ ' + str(np.round(np.std(auc_rounds_nn) / np.sqrt(len(np.array(auc_rounds_nn).ravel())), decimals = decimals)) +\
              cur_p_value_add_str + cur_p_value_model_str + '\\\\')
        if 'w/o pre-training' not in cur_model:
            better_random_pvalue = ttest_1samp(a=np.array(auc_rounds_rf).ravel(),popmean=.5,alternative = 'greater').pvalue
            if better_random_pvalue < 0.05:
                cur_p_value_add_str = '*'
            else:
                cur_p_value_add_str = ''
            base_values = model_result_dicts[cur_model.replace('(CP SP-EyeGAN) fine-tuning','(w/o pre-training)')]
            tt_test_pvalue = ttest_ind(base_values,np.array(auc_rounds_rf).ravel(),alternative='two-sided').pvalue
            if tt_test_pvalue < 0.05 and np.mean(base_values) < np.mean(auc_rounds_rf):
                cur_p_value_model_str += '$\dagger$'
            else:
                cur_p_value_model_str += ''
            print('    ' + cur_model.replace('fine-tuning','random forest') + ' & ' + str(np.round(np.mean(auc_rounds_rf), decimals = decimals)) +\
                 ' $\pm$ ' + str(np.round(np.std(auc_rounds_rf) / np.sqrt(len(np.array(auc_rounds_rf).ravel())), decimals = decimals)) +\
                  cur_p_value_add_str + cur_p_value_model_str + '\\\\')

evaluate Video_Fractals
    EKYT (w/o pre-training) & 0.552 $\pm$ 0.026*\\
    EKYT (CP SP-EyeGAN) fine-tuning & 0.516 $\pm$ 0.028\\
    EKYT (CP SP-EyeGAN) random forest & 0.606 $\pm$ 0.025*\\
    CLRGaze (w/o pre-training) & 0.49 $\pm$ 0.026\\
    CLRGaze (CP SP-EyeGAN) fine-tuning & 0.514 $\pm$ 0.023\\
    CLRGaze (CP SP-EyeGAN) random forest & 0.626 $\pm$ 0.024*$\dagger$\\
evaluate Video_The_Present
    EKYT (w/o pre-training) & 0.49 $\pm$ 0.016\\
    EKYT (CP SP-EyeGAN) fine-tuning & 0.43 $\pm$ 0.017\\
    EKYT (CP SP-EyeGAN) random forest & 0.375 $\pm$ 0.015\\
    CLRGaze (w/o pre-training) & 0.494 $\pm$ 0.017\\
    CLRGaze (CP SP-EyeGAN) fine-tuning & 0.402 $\pm$ 0.016\\
    CLRGaze (CP SP-EyeGAN) random forest & 0.368 $\pm$ 0.015\\
evaluate Video_Despicable_Me
    EKYT (w/o pre-training) & 0.544 $\pm$ 0.011*\\
    EKYT (CP SP-EyeGAN) fine-tuning & 0.522 $\pm$ 0.011*\\
    EKYT (CP SP-EyeGAN) random forest & 0.579 $\pm$ 0.011*$\dagger$\\
    CLRGaze (w/o pre-training) & 0.562 $\